In [426]:
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [427]:
import pickle
import os

In [428]:
def get_var_name(var_value, namespace):
    """
    Given a variable value and namespace (e.g. globals() or locals()),
    returns the name of the variable as a string.
    """
    for name in namespace:
        if namespace[name] is var_value:
            return name
    return None

def remove_field(element):
    element.send_keys(Keys.CONTROL + 'A')
    element.send_keys(Keys.DELETE)

In [429]:
path = 'driver/chromedriver.exe'
driver = webdriver.Chrome(path)

In [430]:
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [431]:
driver.get('https://www.jobstreet.co.id/');
time.sleep(2)

Search Bar

In [469]:
job_name = 'akuntansi'

job_title = '-'.join(job_name.split())
driver.get(f'https://www.jobstreet.co.id/id/job-search/{job_title}-jobs-in-indonesia/')
driver.implicitly_wait(20)

jobs = []
job_num = 1

In [470]:
while True:
    job_cards = driver.find_elements_by_css_selector('[data-search-sol-meta]')
    # job_cards = driver.find_elements_by_tag_name('article')
    curr_page = int(driver.find_element_by_id('pagination').get_attribute('value'))
    driver.implicitly_wait(2)

    for job_card in job_cards:
        job = {}

        # scroll
        driver.execute_script('arguments[0].scrollIntoView({block: "end"});', job_card)

        try:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        except:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        

        '''scraping'''
        # job header: list of header information
        try:
            job_header = driver.find_element_by_css_selector('[data-automation="jobDetailsHeader"]').text.split('\n')[:3]
            driver.implicitly_wait(5)

            job['Title'], job['Company'], job['Area'] = job_header
        except:
            print('❌ JobTitle raise exception!')
            job['Title'], job['Company'], job['Area'] = None, None, None

        # job detail object
        try:
            job_detail = driver.find_element_by_css_selector('[data-automation="splitModeJobDetailsScrollWrapper"]')
            driver.implicitly_wait(5)

            try:
                # job description: list of description
                job_description = driver.find_element_by_css_selector('[data-automation="jobDescription"]').text.split('\n')
                driver.implicitly_wait(5)

                job['Description'] = job_description
            except:
                print('❌ JobDescription raise exception!')
                job['Description'] = None

            try:
                # additional information: list of information
                job_additional_information = job_detail.find_element_by_xpath('div/div[2]/div/div[1]/div/div[2]').text.split('\n')[1:][1::2]
                driver.implicitly_wait(5)

                job_additional_information = [i for i in job_additional_information if i.lower() != 'tidak terspesifikasi']

                job['Information'] = job_additional_information
            except:
                print('❌ JobInformation raise exception!')
                job['Information'] = None
        except:
            print('❌ JobDescription and JobInformation raise exception!')
            job['Description'], job['Information'] = None, None

        jobs.append(job)

        print(job_num, f'📖{curr_page}📖', job['Title'], 'scraped successfully! 👍')

        job_num += 1
    
    # scroll to "lanjut"
    driver.execute_script('arguments[0].scrollIntoView({block: "center"});', driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]"))
    # go to the next page
    driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]").click()
    driver.implicitly_wait(10)

    next_page = int(driver.find_element_by_id('pagination').get_attribute('value'))

    if curr_page == next_page:
        print('> raise break <')
        break


1 📖1📖 Accounting scraped successfully! 👍
2 📖1📖 Accounting Staff scraped successfully! 👍
3 📖1📖 ACCOUNTING STAFF scraped successfully! 👍
4 📖1📖 STAFF ACCOUNTING scraped successfully! 👍
5 📖1📖 Accounting & Tax Staff scraped successfully! 👍
6 📖1📖 Admin Accounting scraped successfully! 👍
7 📖1📖 ACCOUNTING STAFF scraped successfully! 👍
8 📖1📖 Staff Accounting scraped successfully! 👍
9 📖1📖 FINANCE & ACCOUNTING STAFF scraped successfully! 👍
10 📖1📖 Accounting Staff scraped successfully! 👍
11 📖1📖 ACCOUNTING STAFF scraped successfully! 👍
12 📖1📖 Accounting Staff scraped successfully! 👍
13 📖1📖 Finance accounting staff scraped successfully! 👍
14 📖1📖 Staff Account Payable scraped successfully! 👍
15 📖1📖 Accounting Staff Jakarta Pusat scraped successfully! 👍
16 📖1📖 Accounting Staff scraped successfully! 👍
17 📖1📖 Accounting Staff scraped successfully! 👍
18 📖1📖 Junior Accounting Staff scraped successfully! 👍
19 📖1📖 Finance and Accounting Staff scraped successfully! 👍
20 📖1📖 Accounting Staff scraped successfu

In [471]:
len(jobs)

3198

In [473]:
filename = 'jobstreet_' + ''.join(job_name.title().split())

with open(f'data/scraped/{filename}.pickle', 'wb') as file:
    pickle.dump(jobs, file)

In [474]:
# with open("data/scraped/jobstreet_Akuntansi.pickle", "rb") as f:
#     my_list = pickle.load(f)

# print(len(my_list))

3198


In [232]:
# body = driver.find_element_by_tag_name("body")
# body.send_keys(Keys.CONTROL + '-')

In [233]:
# job_search = driver.find_element_by_css_selector('#global-nav-typeahead > input')

# job_search.click()
# job_search.send_keys(Keys.CONTROL + 'a')
# job_search.send_keys(Keys.DELETE)
# job_search.send_keys('ios developer')

# location_search = driver.find_element_by_css_selector('#jobs-search-box-location-id-ember1360')

# location_search.click()
# location_search.send_keys(Keys.CONTROL + 'a')
# location_search.send_keys(Keys.DELETE)
# location_search.send_keys('indonesia')

# search_button = driver.find_element_by_xpath('//*[@id="global-nav-search"]/div/div[2]/button[1]')
# search_button.click()

# actions = ActionChains(driver)
# actions.send_keys(Keys.ENTER)
# actions.perform()